In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing,svm
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
raw_df=pd.read_csv("../input/train.csv")

def gini(actual, pred, cmpcol = 0, sortcol = 1):
     assert( len(actual) == len(pred) )
     all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
     all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
     totalLosses = all[:,0].sum()
     giniSum = all[:,0].cumsum().sum() / totalLosses
     giniSum -= (len(actual) + 1) / 2.
     return giniSum / len(actual)
 
def gini_normalized(a, p):
     return gini(a, p) / gini(a, a)


# Prepare column names 

In [ ]:
# select functions
def is_bin(s):
    return ("bin" in s)
def is_cat(s):
    return ("cat" in s)
def is_val(s):
    return not (is_bin(s) or is_cat(s))

# filter column names
cols = list(raw_df.columns)[2:]
cat_cols=list(filter(is_cat, cols))
bin_cols=list(filter(is_bin, cols))
val_cols=list(filter(is_val, cols))


# Prepare columns

In [ ]:
# preparing binary columns
bin_df=raw_df[bin_cols]

# preparing value columns
raw_val_df=raw_df[val_cols]
val_scaler = preprocessing.StandardScaler()
val_scaler.fit(raw_val_df)
val_df=pd.DataFrame(val_scaler.transform(raw_val_df))

# preparing category columns
raw_cat_df=raw_df[cat_cols]
cat_imputer = preprocessing.Imputer(missing_values=-1, strategy='most_frequent', axis=0)
cat_imputer.fit(raw_cat_df)
raw_cat_ne_df = cat_imputer.transform(raw_cat_df)
cat_one_hot_encoder = preprocessing.OneHotEncoder(sparse=False)
cat_one_hot_encoder.fit(raw_cat_ne_df)
cat_df = pd.DataFrame(cat_one_hot_encoder.transform(raw_cat_ne_df))

ans_df = raw_df[['target']]
# join dfs
df = ans_df.join(bin_df.join(cat_df.join(val_df, lsuffix="_cat",rsuffix="_val"),lsuffix="_bin",rsuffix="_nbi"))




In [ ]:
train_df, cv_df = np.split(df.sample(frac=1), [int(.8*len(df))])
feat_cols = list(train_df.columns)[1:]
train_feat=train_df[feat_cols]
train_ans=train_df[["target"]]
# Any results you write to the current directory are saved as output.

In [ ]:
clf1=lgb.LGBMClassifier(n_estimators=400,learning_rate=0.02)
clf1.fit(train_feat,train_ans.values.ravel())
clf2=lgb.LGBMClassifier(n_estimators=600,learning_rate=0.02)
clf2.fit(train_feat,train_ans.values.ravel())
clf3=lgb.LGBMClassifier(n_estimators=800,learning_rate=0.02)
clf3.fit(train_feat,train_ans.values.ravel())

In [ ]:
cv_feat=cv_df[feat_cols]
cv_ans=cv_df[["target"]]
#print(clf.score(cv_feat,cv_ans.values.ravel()))
ans1=clf1.predict_proba(cv_feat)[:,1]
anst1=clf1.predict_proba(train_feat)[:,1]
ans2=clf2.predict_proba(cv_feat)[:,1]
anst2=clf2.predict_proba(train_feat)[:,1]
ans3=clf3.predict_proba(cv_feat)[:,1]
anst3=clf3.predict_proba(train_feat)[:,1]
print(gini_normalized(cv_ans.values.ravel(),ans1))
print(gini_normalized(train_ans.values.ravel(),anst1))
print(gini_normalized(cv_ans.values.ravel(),ans2))
print(gini_normalized(train_ans.values.ravel(),anst2))
print(gini_normalized(cv_ans.values.ravel(),ans3))
print(gini_normalized(train_ans.values.ravel(),anst3))
ans=(ans1+ans2+ans3)/(3.0)
anst=(anst1+anst2+anst3)/(3.0)
print(gini_normalized(cv_ans.values.ravel(),ans))
print(gini_normalized(train_ans.values.ravel(),anst))
ans=(ans1*ans2*ans3)
anst=(anst1*anst2*anst3)
print(gini_normalized(cv_ans.values.ravel(),ans))
print(gini_normalized(train_ans.values.ravel(),anst))
ans=np.maximum.reduce([ans1,ans2,ans3])
anst=np.maximum.reduce([anst1,anst2,anst3])
print(gini_normalized(cv_ans.values.ravel(),ans))
print(gini_normalized(train_ans.values.ravel(),anst))
ans=np.minimum.reduce([ans1,ans2,ans3])
anst=np.minimum.reduce([anst1,anst2,anst3])
print(gini_normalized(cv_ans.values.ravel(),ans))
print(gini_normalized(train_ans.values.ravel(),anst))
#print(normalized_gini(clf.predict_proba(cv_feat),cv_ans.values.ravel()))

In [ ]:
raw_df=pd.read_csv("../input/test.csv")
# preparing binary columns
bin_df=raw_df[bin_cols]

# preparing value columns
raw_val_df=raw_df[val_cols]
val_df=pd.DataFrame(val_scaler.transform(raw_val_df))

# preparing category columns
raw_cat_df=raw_df[cat_cols]
raw_cat_ne_df = cat_imputer.transform(raw_cat_df)
cat_df = pd.DataFrame(cat_one_hot_encoder.transform(raw_cat_ne_df))

#ans_df = raw_df[['target']]
# join dfs
df = bin_df.join(cat_df.join(val_df, lsuffix="_cat",rsuffix="_val"),lsuffix="_bin",rsuffix="_nbi")


In [ ]:
pred1 = clf1.predict_proba(df)
pred2 = clf2.predict_proba(df)
pred3 = clf3.predict_proba(df)
print(df.shape)
print(df.head())

In [ ]:
(raw_df[['id']].join(pd.DataFrame(pred1[:,1], columns=['target']))).to_csv("./outie1.csv",index=False)
(raw_df[['id']].join(pd.DataFrame(pred2[:,1], columns=['target']))).to_csv("./outie2.csv",index=False)
pred=pred1[:,1]+pred2[:,1]+pred3[:,1]
pred=pred/3.0
(raw_df[['id']].join(pd.DataFrame(pred, columns=['target']))).to_csv("./outie3.csv",index=False)
pred=pred1[:,1]*pred2[:,1]*pred3[:,1]
(raw_df[['id']].join(pd.DataFrame(pred, columns=['target']))).to_csv("./outie4.csv",index=False)
pred=np.maximum.reduce([pred1[:,1],pred2[:,1],pred3[:,1]])
(raw_df[['id']].join(pd.DataFrame(pred, columns=['target']))).to_csv("./outie5.csv",index=False)
pred=np.minimum.reduce([pred1[:,1],pred2[:,1],pred3[:,1]])
(raw_df[['id']].join(pd.DataFrame(pred, columns=['target']))).to_csv("./outie6.csv",index=False)